In [1]:
%load_ext autoreload
%autoreload 2

# Import modules

In [2]:
import numpy as np
import torch
from IPython.display import Audio

from musicvae.music_utils import numpy_to_midi, PITCH_ENCODED
from musicvae.data_utils import parse_one_hot_encode
from musicvae.train import train_model

# Train and return model

In [3]:
models = train_model("datasets/train.npy", "datasets/validation.npy")

Epoch : 5 (7.09s) - train_loss : 258.44 train_acc : 0.32 val_loss : 270.96 val_acc : 0.29
Epoch : 10 (7.15s) - train_loss : 218.85 train_acc : 0.37 val_loss : 247.54 val_acc : 0.29
Epoch : 15 (6.95s) - train_loss : 218.12 train_acc : 0.37 val_loss : 249.60 val_acc : 0.29
Epoch : 20 (7.13s) - train_loss : 214.40 train_acc : 0.36 val_loss : 248.41 val_acc : 0.29
Epoch : 25 (6.99s) - train_loss : 218.81 train_acc : 0.37 val_loss : 248.96 val_acc : 0.29
Epoch : 30 (7.17s) - train_loss : 214.14 train_acc : 0.37 val_loss : 249.12 val_acc : 0.29
Epoch : 35 (6.78s) - train_loss : 218.88 train_acc : 0.35 val_loss : 247.92 val_acc : 0.29
Epoch : 40 (6.87s) - train_loss : 212.66 train_acc : 0.37 val_loss : 263.17 val_acc : 0.29
Epoch : 45 (6.93s) - train_loss : 222.44 train_acc : 0.33 val_loss : 261.64 val_acc : 0.12
Epoch : 50 (6.91s) - train_loss : 214.84 train_acc : 0.37 val_loss : 251.70 val_acc : 0.29


# Do a prediction on a test data

In [4]:
test_dataset = np.load("datasets/test.npy")

sample = np.expand_dims(test_dataset[11], axis=0)

In [5]:
midi = numpy_to_midi(parse_one_hot_encode(sample[0]), 10)
Audio(midi.fluidsynth(fs=44100), rate=44100)

fluidsynth: error: Unknown integer parameter 'synth.sample-rate'


In [6]:
encoder, conductor, decoder = models

In [7]:
encoder.eval()
conductor.eval()
decoder.eval()

with torch.no_grad():
    test_data = torch.from_numpy(sample).to(torch.float32)

    z, _, _ = encoder(test_data)
    bar_feature = conductor(z)
    music_notes_prob = decoder(bar_feature)
    _, idx = torch.max(music_notes_prob, dim=2)
    mask = torch.arange(music_notes_prob.size(2)).reshape(
        1, 1, -1
    ) == idx.unsqueeze(2)
    music_notes_pred = torch.zeros_like(music_notes_prob)
    music_notes_pred[mask] = 1.0

predicted = music_notes_pred.numpy()

In [ ]:
midi = numpy_to_midi(parse_one_hot_encode(predicted)[0], 10)
Audio(midi.fluidsynth(fs=44100), rate=44100)